# Week 11 Coding Homework – Event Studies & Placebo Tests

In [ ]:
import numpy as np
import statsmodels.api as sm

# For reproducibility in ad-hoc experiments
np.random.seed(0)

num = 1000
event_time = int(num / 2)


## Question 1 – Power to detect the event at `event_time + 1`

We simulate many datasets using the provided code pattern, compute the t-statistic
at `event_time + 1`, and estimate the probability that `|t| > 1.96` (i.e., the
event is detected at the 5% significance level).

In [ ]:
def simulate_q1_once(num=1000):
    event_time = int(num / 2)
    
    # Generate data
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num
    R_target = (
        2
        + R_market
        + np.random.normal(0, 1, num)
        + (np.arange(num) == int(num / 2) + 1) * 2
    )
    
    # Fit market model before event_time
    results = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit()
    
    # Residuals over full horizon
    resid = R_target - results.predict(sm.add_constant(R_market))
    
    # t-stat at event_time + 1
    t_val = resid[event_time + 1] / resid[:event_time].std(ddof=2)
    return t_val

# Run many simulations
np.random.seed(0)
num_sims = 2000
t_values = [simulate_q1_once(num) for _ in range(num_sims)]

# Estimate power (probability of detection)
power_estimate = np.mean(np.abs(t_values) > 1.96)
power_estimate


## Question 2 – Placebo tests on a fixed dataset

We now:
1. Generate a **single** dataset (fixed).
2. For each possible fictitious `event_time`, train the market model **only on data before** that fictitious time.
3. Compute the t-statistic at the fictitious event time.
4. Check what fraction of these placebo tests exceed `|t| > 1.96`.

With a well-specified model and independent errors, this should be close to the 5% nominal level.


In [ ]:
# Generate one fixed dataset
np.random.seed(0)
R_market = np.random.normal(0, 1, num) + np.arange(num) / num
R_target = (
    2
    + R_market
    + np.random.normal(0, 1, num)
    + (np.arange(num) == event_time + 1) * 2  # real event at event_time+1
)

# Perform placebo tests for all plausible fictitious event times
placebo_results = []

# Avoid too-small samples at the beginning and end
for fict_event in range(10, num - 10):
    # Train on data BEFORE the fictitious event
    R_m_train = R_market[:fict_event]
    R_t_train = R_target[:fict_event]
    
    model = sm.OLS(R_t_train, sm.add_constant(R_m_train)).fit()
    
    # Residuals on full series
    resid = R_target - model.predict(sm.add_constant(R_market))
    
    # t-stat at fictitious event
    t_stat = resid[fict_event] / resid[:fict_event].std(ddof=2)
    
    placebo_results.append(np.abs(t_stat) > 1.96)

fraction_placebo_detect = np.mean(placebo_results)
fraction_placebo_detect


## Question 3 – Local placebo tests around the true event

Now we:
1. Generate many *different* datasets (by changing the seed).
2. In each dataset, compute the t-statistic at the **true** event time (`event_time + 1`).
3. Also compute t-statistics for 20 fictitious events **before** and 20 **after** the real event.
4. For each dataset, compute what fraction of these 40 placebo t-statistics are **larger in absolute value** than the true event's t-stat.
5. Average that fraction across many runs.


In [ ]:
def simulate_q3_once(num=1000, seed=0):
    np.random.seed(seed)
    event_time = int(num / 2)
    
    # Generate dataset
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num
    R_target = (
        2
        + R_market
        + np.random.normal(0, 1, num)
        + (np.arange(num) == event_time + 1) * 2  # real event
    )
    
    # Fit on data before the true event_time (as in Q1)
    results = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit()
    resid = R_target - results.predict(sm.add_constant(R_market))
    
    # True event t-stat
    t_true = resid[event_time + 1] / resid[:event_time].std(ddof=2)
    
    # Placebo events: 20 before and 20 after the true event
    placebo_ts = []
    for offset in range(-20, 0):
        fict_event = event_time + 1 + offset
        t_val = resid[fict_event] / resid[:fict_event].std(ddof=2)
        placebo_ts.append(t_val)
    for offset in range(1, 21):
        fict_event = event_time + 1 + offset
        t_val = resid[fict_event] / resid[:fict_event].std(ddof=2)
        placebo_ts.append(t_val)
    
    placebo_ts = np.array(placebo_ts)
    frac_larger = np.mean(np.abs(placebo_ts) > np.abs(t_true))
    return frac_larger

# Run many simulations with different seeds
num_runs = 300
fractions = [simulate_q3_once(num=num, seed=seed) for seed in range(num_runs)]
np.mean(fractions)


## Question 4 – Placebo tests with autocorrelated errors

Now we repeat the **Question 2** placebo procedure, but instead of using
independent normal errors for `R_target`, we generate **autocorrelated errors**
using the provided `make_error` function with `corr_const = 0.9`.

Correlated errors tend to create long runs of similarly signed residuals, which
inflates the probability of false positives in the placebo tests.


In [ ]:
def make_error(corr_const, num):
    sigma = 5 * 1 / np.sqrt((1 - corr_const)**2 / (1 - corr_const**2))
    err = []
    prev = np.random.normal(0, sigma)
    for n in range(num):
        prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, sigma)
        err.append(prev)
    return np.array(err)

# Generate one fixed dataset with autocorrelated errors
np.random.seed(0)
R_market = np.random.normal(0, 1, num) + np.arange(num) / num
err = make_error(0.9, num)

R_target_auto = (
    2
    + R_market
    + err
    + (np.arange(num) == event_time + 1) * 2  # real event at event_time+1
)

# Placebo tests over many fictitious event times
placebo_results_auto = []

for fict_event in range(10, num - 10):
    # Train on data before fictitious event
    R_m_train = R_market[:fict_event]
    R_t_train = R_target_auto[:fict_event]
    
    model = sm.OLS(R_t_train, sm.add_constant(R_m_train)).fit()
    
    resid = R_target_auto - model.predict(sm.add_constant(R_market))
    
    t_stat = resid[fict_event] / resid[:fict_event].std(ddof=2)
    placebo_results_auto.append(np.abs(t_stat) > 1.96)

fraction_placebo_detect_auto = np.mean(placebo_results_auto)
fraction_placebo_detect_auto
